In [ ]:
CSV_FILE = "/home/ttuser/git/tt-metal/generated/profiler/reports/2025_07_27_19_45_54/ops_perf_results_2025_07_27_19_45_54.csv"

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact


In [ ]:
# Load the CSV file into a pandas DataFrame
df = pd.read_csv(CSV_FILE)

# Rename the columns for clarity
df = df.rename(columns={
    'OP CODE': 'Operation',
    'HOST DURATION [ns]': 'Host Time',
    'OP TO OP LATENCY [ns]': 'Time Between Ops',
    'DEVICE FW DURATION [ns]': 'Device Time'
})


# Filter out rows before compilation finished
mask = (df['Operation'] == 'ProfilerNoopOperation') & df['ATTRIBUTES'].str.contains('compilation_finished', na=False)
matching_indices = df.index[mask]
assert not matching_indices.empty, "No 'compilation_finished' found in ProfilerNoopOperation attributes"
latest_compilation_flag = matching_indices[-1]
df = df.iloc[latest_compilation_flag + 1:]

# Find number of training steps
mask = (df['Operation'] == 'ProfilerNoopOperation') & df['ATTRIBUTES'].str.contains('iteration_', na=False)
matching_indices = df.index[mask]
num_training_steps = 3 # for backward compatibility, will be removed before merge
if not matching_indices.empty:
    filtered_df = df[df.index.isin(matching_indices)]
    num_training_steps = len(filtered_df['ATTRIBUTES'].unique())

df = df[df['Operation'] != 'ProfilerNoopOperation']

all_operations = df['Operation'].unique()

num_training_steps

In [ ]:
def draw_diagrams_with_aggregation(aggregation):
    grouped = df.groupby('Operation').agg({
        'Host Time': aggregation,
        'Time Between Ops': aggregation,
        'Device Time': aggregation
    })

    time_columns = ['Time Between Ops', 'Device Time', 'Host Time']
    topk = 15

    for col in time_columns:
        # ----- top-k + “Others” slice -------------------------------------
        sorted_times = grouped[col].sort_values(ascending=False)
        top_times    = sorted_times.head(topk)
        others_sum   = sorted_times.iloc[topk:].sum()
        if others_sum:
            top_times = pd.concat([top_times, pd.Series({'Others': others_sum})])

        labels = top_times.index.tolist()
        sizes  = top_times.values
        pct    = 100 * sizes / sizes.sum()

        # ----- plot -------------------------------------------------------
        fig, ax = plt.subplots(figsize=(8, 8))
        wedges, _ = ax.pie(sizes, startangle=140)   # no autopct ⇒ nothing on pie

        legend_text = [f'{lbl} — {p:.1f}%' for lbl, p in zip(labels, pct)]
        ax.legend(
            wedges,
            legend_text,
            title=f'Operations (share of {aggregation})',
            loc='center left',
            bbox_to_anchor=(1, 0.5)
        )

        ax.set_title(f'Top {topk} Operations by {aggregation} {col}')
        ax.axis('equal')
        plt.show()

        # display(fig)    # ➋ show it once

In [ ]:
draw_diagrams_with_aggregation('sum')

In [ ]:
draw_diagrams_with_aggregation('mean')

In [ ]:
def name_per_aggregation(aggregation):
    if aggregation == 'sum':
        return 'Total (per training step)'
    elif aggregation == 'mean':
        return 'Average'
    else:
        raise ValueError(f"Unsupported aggregation: {aggregation}")
    
def draw_charts_with_aggregation(aggregation):
    grouped = df.groupby('Operation').agg({
        'Host Time': aggregation,
        'Time Between Ops': aggregation,
        'Device Time': aggregation
    })

    time_columns = ['Time Between Ops', 'Device Time', 'Host Time']

    # Loop through each time column to create horizontal bar charts and pie charts
    for col in time_columns:
        # Extract the total times per operation for the current column
        total_times = grouped[col] / 1_000_000  # Convert from nanoseconds to milliseconds
        if aggregation == 'sum':
            total_times /= num_training_steps  # Normalize by number of training steps if aggregation is 'sum'
        
        # Create a horizontal bar chart
        plt.figure(figsize=(10, 6))
        total_times.sort_values().plot(kind='barh', color='skyblue') 
        plt.title(f'{name_per_aggregation(aggregation)} {col} per Operation')
        plt.xlabel(f'{name_per_aggregation(aggregation)} {col} (ms)')
        plt.ylabel('Operation')
        plt.tight_layout()
        plt.show()

In [ ]:
draw_charts_with_aggregation('mean')

In [ ]:
draw_charts_with_aggregation('sum')

In [ ]:
@interact(operation=all_operations)
def draw_per_operation_stats(operation):
    df_op = df[df['Operation'] == operation]
    if df_op.empty:
        print(f"No data available for operation: {operation}")
        return

    metrics = [
        ('Host Time (ms)',          df_op['Host Time']        / 1_000_000),
        ('Time Between Ops (ms)',   df_op['Time Between Ops'] / 1_000_000),
        ('Device Time (ms)',        df_op['Device Time']      / 1_000_000),
    ]

    for title, series in metrics:
        # --- build figure explicitly so we know which one to close ------------
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.plot(series.index, series.values, marker='o', color='red')
        ax.set_title(f'{operation} – {title}')
        ax.set_xlabel('Index')
        ax.set_ylabel(title)
        ax.grid(True)
        fig.tight_layout()
        plt.show()

In [ ]:
# example how to manually extract performance data for a specific operation
e = df[df['Operation'] == 'Untilize']
e = e[['Operation', 'Host Time', 'Time Between Ops', 'Device Time']]
e

In [ ]:
metrics = ['Host Time', 'Time Between Ops', 'Device Time']

def anomaly_detection_per_operation(operation, metric):
    df_op = df[df['Operation'] == operation]
    if df_op.empty:
        print(f"No data available for operation: {operation}")
        return

    # Calculate the mean and standard deviation for each metric
    
    series = df_op[metric]
    
    mean = series.mean()
    std_dev = series.std()
    
    # Identify anomalies as points that are more than 3 standard deviations from the mean
    anomalies = series[(series < mean - 3 * std_dev) | (series > mean + 3 * std_dev)]
    
    if not anomalies.empty:
        return df_op.loc[anomalies.index]
    return None

In [ ]:
import json

def is_not_nan(value):
    """Check if a value is not NaN."""
    return pd.notna(value) and value != 'NaN' and value != 'nan' and value != ''

@interact(operation=all_operations, metric_name=metrics)
def show_anomalies_attributes_per_metric(operation, metric_name):
    anomaly_df = anomaly_detection_per_operation(operation, metric_name)

    if anomaly_df is None:
        print(f"No anomalies detected for {operation} in {metric_name}.")
        return

    for index, row in anomaly_df.iterrows():
        attr = row['ATTRIBUTES']
        core_count = row['CORE COUNT']
        metric_value = row[metric_name]

        # find all columns with prefix `INPUT_`
        input_columns = [col for col in row.index if col.startswith('INPUT_')]
        input_values = {col: row[col] for col in input_columns if is_not_nan(row[col])}

        # improve print of dictionary with json
        input_values = json.dumps(input_values, indent=8)

        if isinstance(attr, str):
            attr = attr.replace(';', ',')
            attr = attr.replace('\'', '"')
            try:
                attr = json.loads(attr)
            except:
                pass
            attr = json.dumps(attr, indent=8) if isinstance(attr, dict) else attr

        print(f"Anomaly at index {index}: ")
        print(f"    {metric_name} = {metric_value / 1_000_000} ms")
        print(f"    core count = {core_count}")
        print(f"    inputs = {input_values}")
        print(f"    attributes = {attr}")
